In [ ]:
!pip install -q torch-cluster 
!pip install -q torch-scatter
!pip install -q torch-sparse
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 38.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Read data
import os
import pandas as pd
import networkx as nx
import matplotlib as mpl
import numpy as np
import random

PRJ_DIR = os.path.join('/','content','drive','MyDrive','Colab Notebooks','eliorc_node2vec')
DATA_DIR = os.path.join(PRJ_DIR,'Facebook-Social-Network-Analysis-master')

# Read data
fb = pd.read_csv(os.path.join(DATA_DIR,'facebook_combined.txt'), delim_whitespace=True, names=['Source', 'Destination'])
fb.head()

,Source,Destination
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [ ]:
#contents of one such file, namely '0.featnames'
with open(os.path.join(DATA_DIR,'facebook','0.featnames'), 'r') as f:
        feature_names = f.readlines()
        
feature_names_df = pd.read_csv(os.path.join(DATA_DIR,'facebook','0.featnames'), names=['Feature Names'])
# format of the values in the file : 
# 'feature number' + ' ' + 'feature name' + ';' + 'feature type'
# feature type always starts with 'anonymized feature' 
feature_names_df

,Feature Names
0,0 birthday;anonymized feature 0
1,1 birthday;anonymized feature 1
2,2 birthday;anonymized feature 2
3,3 birthday;anonymized feature 3
4,4 birthday;anonymized feature 4
...,...
219,219 work;start_date;anonymized feature 170
220,220 work;start_date;anonymized feature 171
221,221 work;start_date;anonymized feature 203
222,222 work;start_date;anonymized feature 204


In [ ]:
# printing list of unique features in the '0.featnames'
unique_features = []

for i in range(len(feature_names)):
  feature_names[i] = feature_names[i].split(' ', 1)[1]
  feature_names[i] = feature_names[i].rsplit(';' , 1)[0]
  unique_features.append(feature_names[i])

list(set(unique_features))

['work;end_date',
 'education;type',
 'education;with;id',
 'work;location;id',
 'birthday',
 'first_name',
 'education;degree;id',
 'gender',
 'work;employer;id',
 'education;concentration;id',
 'languages;id',
 'work;start_date',
 'work;with;id',
 'hometown;id',
 'location;id',
 'education;year;id',
 'education;school;id',
 'education;classes;id',
 'locale',
 'work;position;id',
 'last_name']

In [ ]:
#@title Creating a nested dictionary of all the nodes along with its attributes
def ExtractFeatureNames(path):
    with open(path, 'r') as f:
        feature_names = f.readlines()

    for i in range(len(feature_names)):
        feature_names[i] = feature_names[i].split(' ', 1)[1]
        feature_names[i] = feature_names[i].rsplit('\n' , 1)[0]
        feature_names[i] = feature_names[i].rsplit(';' , 1)
        feature_names[i][1] = feature_names[i][1].rsplit(' ' , 1)[1]
        
    return feature_names

#storing all the numbers of different files in a list
l = ['0','107','348','414','686','698','1684','1912','3437','3980']

node_features = []

# extracting features - 'location;id' and 'education;school;id'
for i in l:
    df = pd.read_csv(os.path.join(DATA_DIR,'facebook',i +'.feat'), 
                     delim_whitespace=True, 
                     names=pd.MultiIndex.from_tuples(ExtractFeatureNames(os.path.join(DATA_DIR,'facebook',i +'.featnames')))).T
    node_features.append(df.loc[['location;id','education;school;id']])

node_attributes = {}
for i in node_features:
    a = {j : i[i[j] == 1][j].index.tolist()[:2] for j in i.columns if len(i[i[j] == 1][j].index.tolist())>1 and 
                                                                 i[i[j] == 1][j].index.tolist()[0][0] == 'location;id'}
    node_attributes.update(a)

#creating the nested dictionary
for k,v in node_attributes.items():
    node_attributes[k] = {i[0] : i[1] for i in node_attributes[k]}
    
#sorting the dictionary
node_attributes = {i[0]: i[1] for i in sorted(node_attributes.items())}

In [ ]:
#@title Dropping unwanted nodes from fb dataframe
#dropping the extra nodes from 'fb' that are not in 'node_attributes' 
no_att_source = list(set([i for i in fb.Source]) - set(node_attributes.keys()))
no_att_dest = list(set([i for i in fb.Destination]) - set(node_attributes.keys()))

#finding indices of the above nodes in fb and dropping them
idx = []
for i in no_att_source:
    idx.extend(fb.index[fb.Source == i].tolist())
    
fb.drop(list(set(idx)), axis=0, inplace=True)
    
idx = []    
for i in no_att_dest:    
    idx.extend(fb.index[fb.Destination == i].tolist())
    
fb.drop(list(set(idx)), axis=0, inplace=True)

In [ ]:
#@title Creating graph
#creating a list of node pair tuples from the 'fb' dataframe
existing_edges = [(fb.iloc[i,0],fb.iloc[i,1]) for i in range(fb.shape[0])]

G = nx.Graph()
for i in node_attributes.keys():
    G.add_node(i)

G.add_edges_from(existing_edges)
print("------Before Dropping nodes with low degree (<3)-----")
print('Number of nodes', len(G.nodes))
print('Number of edges', len(G.edges))
print('Average degree', sum(dict(G.degree).values()) / len(G.nodes))

------Before Dropping nodes with low degree (<3)-----
Number of nodes 1447
Number of edges 18476
Average degree 25.536973047684864


In [ ]:
fb = pd.DataFrame(data=list(G.edges()), columns=['Source', 'Destination'])
fb.head()

,Source,Destination
0,2,116
1,2,226
2,2,326
3,3,25
4,3,67


In [ ]:
nodes = pd.concat([fb['Source'],fb['Destination']]).unique()
print(type(nodes), len(nodes))

<class 'numpy.ndarray'> 1418


In [ ]:
#@title Creating graph
#creating a list of node pair tuples from the 'fb' dataframe
existing_edges = [(fb.iloc[i,0],fb.iloc[i,1]) for i in range(fb.shape[0])]

G = nx.Graph()
fb_nodes = pd.concat([fb['Source'],fb['Destination']]).unique()
for i in fb_nodes:
    G.add_node(i)

G.add_edges_from(existing_edges)
print("------Before Dropping nodes with low degree (<3)-----")
print('Number of nodes', len(G.nodes))
print('Number of edges', len(G.edges))
print('Average degree', sum(dict(G.degree).values()) / len(G.nodes))

------Before Dropping nodes with low degree (<3)-----
Number of nodes 1418
Number of edges 18476
Average degree 26.059238363892806


In [ ]:
#@title Creating Adjacency Matrix

nodelist = list(G.nodes())
adj_G = nx.to_numpy_array(G, nodelist = nodelist)
print(str(adj_G.shape)+'\n')
adj_G

(1418, 1418)



array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
non_existing_edges_40000 = []
while(len(non_existing_edges_40000) < 40000):
    source_idx = random.randint(0, adj_G.shape[0]-1)
    destination_idx = random.randint(0, adj_G.shape[1]-1)
    if source_idx == destination_idx or adj_G[source_idx, destination_idx] == 1:
        continue

    if nx.has_path(G, nodelist[source_idx], nodelist[destination_idx]):
        non_existing_edges_40000.extend([(nodelist[source_idx],nodelist[destination_idx])])
    

non_existing_edge_df = pd.DataFrame(data = non_existing_edges_40000, columns =['Source', 'Destination'])
non_existing_edge_df['Connection'] = 0
non_existing_edge_df.head()

,Source,Destination,Connection
0,2868,2232,0
1,1622,1118,0
2,940,1249,0
3,376,2436,0
4,3425,388,0


In [ ]:
def GettingRemoveableEdges(origin_G, origin_graph_df):
    # Create a list of all indices of the node pairs in the fb dataframe,
    # which when removed won’t change the structure of our graph
    
    # create a copy
    temp_graph_df = origin_graph_df.copy()
    temp_G = nx.from_pandas_edgelist(origin_graph_df, "Source", "Destination",
                                    create_using=nx.Graph())
    
    # for storing removable edges
    removable_edges_indices = []
    
    # number of connected components and nodes of G
    ncc = nx.number_connected_components(origin_G)
    number_of_nodes = len(origin_G.nodes)
    
    # for each node pair we will be removing a node pair and creating a new graph,
    # and check if the number of connected components and the number of nodes
    # are the same as the original graph
    for i in origin_graph_df.index.values:
        if len(removable_edges_indices) >= len(origin_graph_df.index)//2:
            break

        if (i % 100) == 0:
            print(f"row {i}: {origin_graph_df.iloc[i, 0]}, {origin_graph_df.iloc[i, 1]}")
        
        temp_G.remove_edge(origin_graph_df.iloc[i, 0], origin_graph_df.iloc[i, 1])
        if len(temp_G.nodes) == number_of_nodes and nx.number_connected_components(temp_G) == ncc:
            #temp_G.remove_edge(origin_graph_df.iloc[i, 0], origin_graph_df.iloc[i, 1])
            removable_edges_indices.append(i)
        else:
            temp_G.add_edge(origin_graph_df.iloc[i, 0], origin_graph_df.iloc[i, 1])

    with open(os.path.join(PRJ_DIR,'fb_removable_edges_indices.txt'), 'w') as f:
        removable_edges_indices = [str(elem.item()) for elem in removable_edges_indices]
        file_lines = "\n".join(removable_edges_indices)
        f.write(file_lines)

    with open(os.path.join(PRJ_DIR,'fb_removable_edges_indices.txt'), 'r') as f:
        removable_edges_indices = f.readlines()
        removable_edges_indices = [int(elem) for elem in removable_edges_indices]

    return removable_edges_indices

if os.path.isfile(os.path.join(PRJ_DIR,'fb_removable_edges_indices.txt')):
    print("Already have file!")
    with open(os.path.join(PRJ_DIR,'fb_removable_edges_indices.txt'), 'r') as f:
        removable_edges_indices = f.readlines()
        removable_edges_indices = [int(elem) for elem in removable_edges_indices]
else:
    print("Not have file yet!")
    removable_edges_indices = GettingRemoveableEdges(G, fb)

Already have file!


In [ ]:
#@title Creating Dataframe of Removable Edges

removeable_edge_df = fb.loc[removable_edges_indices]
removeable_edge_df['Connection'] = 1
 
removeable_edge_df.head()

,Source,Destination,Connection
0,2,116,1
1,2,226,1
3,3,25,1
4,3,67,1
5,3,142,1


In [ ]:
#@title Creating Subgraph
non_existing_edge_df = non_existing_edge_df.append(removeable_edge_df[['Source', 'Destination', 'Connection']],
                ignore_index=True)

edges_G_past = fb.drop(index=removeable_edge_df.index.values)

G_new = nx.from_pandas_edgelist(edges_G_past, "Source", "Destination",
                               create_using=nx.Graph())

print('Number of nodes', len(G_new.nodes))
print('Number of edges', len(G_new.edges))
print('Average degree', sum(dict(G_new.degree).values()) / len(G_new.nodes))

Number of nodes 1418
Number of edges 9238
Average degree 13.029619181946403


<ipython-input-16-b8f500e20bdb>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_existing_edge_df = non_existing_edge_df.append(removeable_edge_df[['Source', 'Destination', 'Connection']],


In [ ]:
from torch_geometric.nn import Node2Vec
import os.path as osp
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, auc, roc_curve, roc_auc_score,confusion_matrix
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

edges_tensor = torch.from_numpy(edges_G_past.to_numpy())
edges_tensor = torch.permute(edges_tensor, (1, 0))

set_of_pq = [0.25, 0.50, 1, 2, 4]

model_pool = {}

for p_value in set_of_pq:
    for q_value in set_of_pq:
        node2vec = Node2Vec(edge_index = edges_tensor,
                    embedding_dim = 128,
                    walk_length = 16,
                    context_size = 10,
                    walks_per_node = 20,
                    p=p_value, q=q_value).to(device)

        loader = node2vec.loader(batch_size=128, shuffle=True, num_workers=4)
        optimizer = torch.optim.Adam(list(node2vec.parameters()), lr=0.01)

        print("-------Training--------")
        for epoch in range(1, 150):
            loss = train(node2vec, loader, optimizer)
            if epoch % 15 == 0:
                print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

        embedding_nodes = node2vec(torch.arange(node2vec.embedding.weight.size()[0], device=device))

        edge_features = [
            (embedding_nodes[i].detach().cpu().numpy() + embedding_nodes[j].detach().cpu().numpy()) 
            for i,j in zip(non_existing_edge_df['Source'], non_existing_edge_df['Destination'])
        ]

        X = np.array(edge_features)  
        y = non_existing_edge_df['Connection']

        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = random.randint(0, 999999))

        #classifier
        random_forest_clf = RandomForestClassifier(n_estimators=100, max_depth=15)
        
        # alternative metric to optimize over grid parameters: AUC
        random_forest_clf.fit(X_train, y_train)
        
        model_pool[f"model_pq:{p_value}_{q_value}"] = random_forest_clf
        model_pool[f"X_test_pq:{p_value}_{q_value}"] = X_test
        model_pool[f"y_test_pq:{p_value}_{q_value}"] = y_test

-------Training--------
Epoch: 15, Loss: 0.7838
Epoch: 30, Loss: 0.7357
Epoch: 45, Loss: 0.7241
Epoch: 60, Loss: 0.7203
Epoch: 75, Loss: 0.7194
Epoch: 90, Loss: 0.7188
Epoch: 105, Loss: 0.7191
Epoch: 120, Loss: 0.7192
Epoch: 135, Loss: 0.7195
-------Training--------
Epoch: 15, Loss: 0.7816
Epoch: 30, Loss: 0.7344
Epoch: 45, Loss: 0.7237
Epoch: 60, Loss: 0.7205
Epoch: 75, Loss: 0.7190
Epoch: 90, Loss: 0.7189
Epoch: 105, Loss: 0.7190
Epoch: 120, Loss: 0.7192
Epoch: 135, Loss: 0.7195
-------Training--------
Epoch: 15, Loss: 0.7808
Epoch: 30, Loss: 0.7337
Epoch: 45, Loss: 0.7233
Epoch: 60, Loss: 0.7202
Epoch: 75, Loss: 0.7191
Epoch: 90, Loss: 0.7188
Epoch: 105, Loss: 0.7192
Epoch: 120, Loss: 0.7195
Epoch: 135, Loss: 0.7194
-------Training--------
Epoch: 15, Loss: 0.7810
Epoch: 30, Loss: 0.7337
Epoch: 45, Loss: 0.7233
Epoch: 60, Loss: 0.7203
Epoch: 75, Loss: 0.7193
Epoch: 90, Loss: 0.7189
Epoch: 105, Loss: 0.7192
Epoch: 120, Loss: 0.7191
Epoch: 135, Loss: 0.7194
-------Training--------
Epoc

In [ ]:
from sklearn.metrics import accuracy_score

for p_value in set_of_pq:
    for q_value in set_of_pq:
        model = model_pool[f"model_pq:{p_value}_{q_value}"]
        X_test = model_pool[f"X_test_pq:{p_value}_{q_value}"]
        y_test = model_pool[f"y_test_pq:{p_value}_{q_value}"]

        predict_proba = model.predict_proba(X_test)[:,1]
        y_pred = model.predict(X_test)
        print(f"--------------Result with p={p_value} and q={q_value}")
        print('Test set AUC: ', roc_auc_score(y_test, predict_proba))
        print('Test set Acc: ', accuracy_score(y_test, y_pred))

--------------Result with p=0.25 and q=0.25
Test set AUC:  0.9666871326057607
Test set Acc:  0.9111291632818846
--------------Result with p=0.25 and q=0.5
Test set AUC:  0.9673217610069036
Test set Acc:  0.909910641754671
--------------Result with p=0.25 and q=1
Test set AUC:  0.9677897327678768
Test set Acc:  0.9103168155970756
--------------Result with p=0.25 and q=2
Test set AUC:  0.9671699339145193
Test set Acc:  0.9087733549959383
--------------Result with p=0.25 and q=4
Test set AUC:  0.968554072583721
Test set Acc:  0.9123476848090983
--------------Result with p=0.5 and q=0.25
Test set AUC:  0.9689768735183903
Test set Acc:  0.9070674248578392
--------------Result with p=0.5 and q=0.5
Test set AUC:  0.9674990138197097
Test set Acc:  0.906336311941511
--------------Result with p=0.5 and q=1
Test set AUC:  0.9681595805420946
Test set Acc:  0.9055239642567019
--------------Result with p=0.5 and q=2
Test set AUC:  0.9677055514064912
Test set Acc:  0.9115353371242892
--------------Re